इस लेख में शामिल जप्टर नोटबुक [अध्याय 4 कोड बेस](https://github.com/datawhalechina/learn-nlp-with-transformers/tree/main/docs/%E7%AF%87%E7%) में है AB%A04-%E4%BD%BF%E7%94%A8Transformers%E8%A7%A3%E5%86%B3NLP%E4%BB%BB%E5%8A%A1).

प्रासंगिक डेटासेट और मॉडल को तुरंत डाउनलोड करने के लिए इस ट्यूटोरियल को सीधे Google कोलाब नोटबुक का उपयोग करके खोलने की अनुशंसा की जाती है।
यदि आप इस नोटबुक को Google कोलैब में खोल रहे हैं, तो आपको ट्रांसफॉर्मर और 🤗डेटासेट लाइब्रेरीज़ को इंस्टॉल करने की आवश्यकता हो सकती हैउन्हें स्थापित करने के लिए निम्नलिखित आदेश।

In [ ]:
# !पिप डेटासेट ट्रांसफार्मर स्थापित करें
# -i https://pypi.tona.tsinghua.edu.cn/simple

यदि आप इस ज्यूपिटर नोटबुक को अपनी स्थानीय मशीन पर खोल रहे हैं, तो कृपया सुनिश्चित करें कि आपके वातावरण में उपरोक्त पुस्तकालयों के नवीनतम संस्करण स्थापित हैं।

आप इस ज्यूपिटर नोटबुक के लिए विशिष्ट पायथन स्क्रिप्ट फ़ाइल [यहां] (https://github.com/huggingface/transformers/tree/master/examples/langage-modeling) पा सकते हैं और आप अपने मॉडल को एक वितरित में फाइन-ट्यून भी कर सकते हैं एकाधिक जीपीयू या टीपीयू का उपयोग करने का तरीका।

# भाषा मॉडल को ठीक करें

इस ज्यूपिटर नोटबुक में, हम बताएंगे कि भाषा मॉडलिंग कार्य का उपयोग करके किसी भी [🤗ट्रांसफॉर्मर्स](https://github.com/huggingface/transformers) मॉडल को कैसे फाइन-ट्यून किया जाए।

यह ट्यूटोरियल दो प्रकार के भाषा मॉडलिंग कार्यों को कवर करेगा:

+ कारण भाषा मॉडलिंग (सीएलएम): मॉडल को वाक्य में अगले स्थान पर चरित्र की भविष्यवाणी करने की आवश्यकता है (बीईआरटी-जैसे मॉडल के डिकोडर और जीपीटी के समान, जहां वर्ण बाएं से दाएं इनपुट हैं यह सुनिश्चित करने के लिए)। धोखा नहीं देता, एक ध्यानमास्क का उपयोग मॉडल को उसके बाद के वर्णों को देखने से रोकने के लिए किया जाता है, उदाहरण के लिए, जब मॉडल वाक्य में स्थिति i+1 पर वर्ण की भविष्यवाणी करने का प्रयास करता है, तो यह मास्क उसे स्थिति i के बाद के वर्णों तक पहुंचने से रोक देगा।

![अनुमान प्रतिनिधित्व कारण भाषा मॉडलिंग कार्य छवि](./images/causal_भाषा_modeling.png)

+ नकाबपोश भाषा मॉडलिंग (एमएलएम): मॉडल को कुछ वर्णों को पुनर्प्राप्त करने की आवश्यकता है जो इनपुट से बाहर "मास्क" किए गए थे (बीईआरटी-जैसे मॉडल के लिए पूर्व-प्रशिक्षण कार्य)।वह मॉडल पूरे वाक्य को देख सकता है, इसलिए मॉडल "\[MASK\]" टैग के पहले और बाद के वर्णों के आधार पर "\[MASK\]" टैग से पहले के चरित्र की भविष्यवाणी कर सकता है।

![विजेट अनुमान नकाबपोश भाषा मॉडलिंग कार्य का प्रतिनिधित्व करता है](images/masked_langage_modeling.png)

आगे, हम बताएंगे कि प्रत्येक कार्य के लिए डेटासेट को आसानी से कैसे लोड और प्रीप्रोसेस किया जाए, और "ट्रेनर" एपीआई का उपयोग करके मॉडल को कैसे ठीक किया जाए।

बेशक, आप इस ज्यूपिटर नोटबुक के पायथन स्क्रिप्ट संस्करण को सीधे भी चला सकते हैंएक वितरित वातावरण या टीपीयू, जो [उदाहरण फ़ोल्डर] (https://github.com/huggingface/transformers/tree/master/examples) में पाया जा सकता है।

## डेटा तैयार करें

निम्नलिखित कार्यों में, हम एक उदाहरण के रूप में [विकीटेक्स्ट 2](https://huggingface.co/datasets/wikitext#data-instances) डेटासेट का उपयोग करेंगे। आप इस डेटासेट को 🤗Datasets लाइब्रेरी के माध्यम से लोड कर सकते हैं:

In [1]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

Reusing dataset wikitext (/Users/niepig/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20)


यदि आपको निम्न त्रुटि मिलती है:
![अनुरोध त्रुटि](images/request_error.png)

समाधान:

मैक उपयोगकर्ता: ```/etc/hosts``` फ़ाइल में ```199.232.68.133 raw.githubusercontent.com``` एक पंक्ति जोड़ें

Windowso उपयोगकर्ता: ```C:\Windows\System32\drivers\etc\hosts``` फ़ाइल में एक पंक्ति ```199.232.68.133 raw.githubusercontent.com`` जोड़ें

निःसंदेह आप उपरोक्त डेटासेट को [हब](https://huggingface.co/datasets) पर सार्वजनिक रूप से उपलब्ध किसी भी डेटासेट से बदल सकते हैं, या अपनी स्वयं की फ़ाइलों का उपयोग कर सकते हैं, बस निम्नलिखित सेल को अनकम्मेंट करें और पथ को उस पथ से बदलें जो आगे ले जाएगा आपके फ़ाइल पथ पर:

In [ ]:
# डेटासेट = लोड_डेटासेट ("टेक्स्ट", डेटा_फ़ाइल्स = {"ट्रेन": path_to_train.txt, "सत्यापन": path_to_validation.txt}

आप सीएसवी या JSON फ़ाइलों से भी डेटासेट लोड कर सकते हैं, अधिक जानकारी के लिए [पूर्ण दस्तावेज़] (https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) देखें।

डेटासेट में वास्तविक तत्व तक पहुंचने के लिए, आपको पहले एक कुंजी का चयन करना होगा और फिर एक इंडेक्स देना होगा:

In [2]:
datasets["train"][10]

{'text': ' The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede

डेटा की संरचना को शीघ्रता से समझने के लिए, नीचे दिया गया फ़ंक्शन डेटासेट से कुछ यादृच्छिक रूप से चयनित उदाहरण प्रदर्शित करेगा।

In [3]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [4]:
show_random_elements(datasets["train"])

,text
0,"On 3 March 1967 , parliament decided to build four short take @-@ off and landing airports along the Helgeland coast between Trondheim and Bodø . Braathens placed an order for a de Havilland Canada DHC @-@ 6 Twin Otter and planned to start the company Braathens STOL . It applied to operate the route without subsidies , but the concession was rejected and granted with subsidies to Widerøe , which had been operating the routes using seaplanes . \n"
1,
2,"Rao Ramesh was cast as a tantrik who helps Gill 's character in the present era . Mumaith Khan was selected for another item number , a remix version of the hit song "" Bangaru Kodipetta "" from Gharana Mogudu ( 1992 ) ; Gharana Mogudu 's music was also composed by M. M. Keeravani . Chiranjeevi made a special appearance after the song , making Magadheera the first film he appeared in after his entry into politics . When Rajamouli suggested the idea of a cameo appearance , Chiranjeevi was initially hesitant till the director narrated the complete sequence and the importance of the song . \n"
3,
4,
5,= = = Total Nonstop Action Wrestling ( 2015 – present ) = = = \n
6,"The Daily Telegraph gave the visual novel the award for "" Best Script "" in its video game awards of 2011 , stating that "" Love 's layered narrative of a high school teacher embroiled in his student ’ s worries goes places most mainstream video games wouldn 't dare . "" \n"
7,
8,
9,


जैसा कि हम देख सकते हैं, कुछ पाठ विकिपीडिया लेखों के पूर्ण पैराग्राफ हैं, जबकि अन्य केवल शीर्षक या खाली पंक्तियाँ हैं।

## कारणात्मक भाषा मॉडलिंग (सीएलएम)

कॉज़ल लैंग्वेज मॉडल (सीएलएम) के लिए, हम पहले डेटासेट में सभी टेक्स्ट प्राप्त करते हैं और उन्हें टोकन देने के बाद उन्हें जोड़ते हैं, फिर, हम उन्हें विशिष्ट अनुक्रम लंबाई के उदाहरणों में विभाजित करेंगे। इस तरह, मॉडल को निरंतर भाग प्राप्त होंगे इस प्रकार पाठ करें:

```
पाठ 1
```
या
```
टेक्स्ट1 का अंत [BOS_TOKEN] टेक्स्ट2 की शुरुआत
```

इस पर निर्भर करते हुए कि क्या वे डेटासेट में कई मूल पाठों को फैलाते हैं, लेबल इनपुट के समान होंगे, लेकिन बाईं ओर स्थानांतरित हो जाएंगे।

इस उदाहरण में, हम उपयोग करेंगे[`distilgpt2`](https://huggingface.co/distilgpt2) मॉडल आप [यहां](https://huggingface.co/models?filter=causal-lm) सूचीबद्ध किसी भी चेकपॉइंट को चुन सकते हैं।

In [5]:
model_checkpoint = "distilgpt2"

मॉडल को प्रशिक्षित करते समय उपयोग की जाने वाली शब्दावली के साथ सभी पाठ को टोकननाइज़ करने के लिए, हमें एक पूर्व-प्रशिक्षित टोकननाइज़र डाउनलोड करना होगा। यह सब ऑटोटोकनाइज़र क्लास के साथ किया जा सकता है:

In [6]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

अब हम अपने सभी टेक्स्ट पर टोकननाइज़र को कॉल कर सकते हैं, जो डेटासेट लाइब्रेरी से मैप विधि का उपयोग करके किया जा सकता है, सबसे पहले, हम एक फ़ंक्शन को परिभाषित करते हैं जो हमारे टेक्स्ट पर टोकननाइज़र को कॉल करता है:

In [7]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

फिर हम प्रीप्रोसेसिंग को तेज करने के लिए बैच=ट्रू और 4 प्रक्रियाओं का उपयोग करके इसे डेटासेट ऑब्जेक्ट में टोकननाइजेशन पर लागू करते हैं, हमें बाद में टेक्स्ट कॉलम की आवश्यकता नहीं होती है, इसलिए हम इसे छोड़ देते हैं।

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

यदि अब हम डेटासेट के एक तत्व को देखते हैं, तो हम देखते हैं कि टेक्स्ट को मॉडल द्वारा आवश्यक इनपुट_आईड्स द्वारा प्रतिस्थापित कर दिया गया है:

In [9]:
tokenized_datasets["train"][1]

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1],
 'input_ids': [796, 569, 18354, 7496, 17740, 6711, 796, 220, 198]}

अगला चरण थोड़ा अधिक कठिन है: हमें सभी पाठों को एक साथ जोड़ना होगा और फिर परिणाम को एक विशिष्ट `ब्लॉक_साइज़` के छोटे हिस्सों में विभाजित करना होगा, ऐसा करने के लिए, हम विकल्प `बैच` के साथ फिर से `मैप` विधि का उपयोग करेंगे =True`। यह विकल्प हमें अलग-अलग संख्या में नमूने लौटाते हुए डेटासेट में उदाहरणों की संख्या को अलग-अलग करने की अनुमति देता है। इस तरह, हम उदाहरणों के एक बैच से नए उदाहरण बना सकते हैं।

सबसे पहले, हमें मॉडल को पूर्व-प्रशिक्षित करते समय उपयोग की जाने वाली अधिकतम लंबाई प्राप्त करने की आवश्यकता हैयदि आपकी वीडियो मेमोरी फट जाती है तो उम लंबाई यहां 128 पर सेट है।

In [10]:
# ब्लॉक_आकार = टोकननाइज़र.मॉडल_मैक्स_लेंथ
block_size = 128

फिर हम अपने टेक्स्ट को समूहीकृत करने के लिए प्रीप्रोसेसिंग फ़ंक्शन लिखते हैं:

In [11]:
def group_texts(examples):
# सभी पाठ को संयोजित करें
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
# हम शेष के अनुरूप भाग को हटा देते हैं, लेकिन यदि मॉडल इसका समर्थन करता है, तो आप आवश्यकतानुसार इस भाग को कस्टमाइज़ कर सकते हैं।
    total_length = (total_length // block_size) * block_size
# max_len द्वारा विभाजित.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

सबसे पहले, ध्यान दें कि हमने लेबल के लिए इनपुट को डुप्लिकेट किया है।

ऐसा इसलिए है क्योंकि 🤗 ट्रांसफार्मर लाइब्रेरी के मॉडल डिफ़ॉल्ट रूप से दाईं ओर शिफ्ट होते हैं, इसलिए हमें इसे मैन्युअल रूप से करने की आवश्यकता नहीं है।

यह भी ध्यान दें कि डिफ़ॉल्ट रूप से, `मैप` विधि प्रीप्रोसेसिंग फ़ंक्शन द्वारा संसाधित करने के लिए 1,000 उदाहरणों का एक बैच भेजेगी, इसलिए यहां, हम शेष को हटा रहे हैं, जिससे प्रत्येक 1,000 उदाहरणों में संयोजित टोकन टेक्स्ट को `ब्लॉक_साइज़` का गुणक बना दिया जाएगा। आप उच्च बैच आकार पास करके इस व्यवहार को बदल सकते हैं (जो कि ओनिश्चित रूप से अधिक धीरे-धीरे संसाधित किया जाना चाहिए)। प्रीप्रोसेसिंग को तेज़ करने के लिए आप `मल्टीप्रोसेसिंग` का भी उपयोग कर सकते हैं:

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

अब हम जाँच सकते हैं कि क्या डेटासेट बदल गया है: अब नमूनों में `ब्लॉक_साइज़` लगातार वर्णों के ब्लॉक हैं, जो संभवतः कई मूल पाठों तक फैले हुए हैं।

In [13]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

' game and follows the " Nameless ", a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven ". \n The game began development in 2010, carrying over a large portion of the work done on Valkyria Chronicles II. While it retained the standard features of the series, it also underwent multiple adjustments, such as making the game more forgiving for series newcomers. Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries, along with Valkyria Chronicles II director Takeshi Oz'

अब जब डेटा साफ हो गया है, तो हम अपने ट्रेनर को इंस्टेंट कर सकते हैं, हम एक मॉडल बनाएंगे:

In [14]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

टॉर्च संस्करण की जाँच करें

In [15]:

import importlib.util
# आयातित आयातलिब_मेटाडेटा
a = importlib.util.find_spec("torch") is not None
print(a)
# _मशाल_संस्करण = आयातलिब_मेटाडेटा.संस्करण("मशाल")
# प्रिंट(_मशाल_संस्करण)

True


और कुछ `प्रशिक्षण तर्क`:

In [16]:
from transformers import Trainer, TrainingArguments

In [17]:
training_args = TrainingArguments(
    "test-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

हम यह सब `ट्रेनर` वर्ग को सौंपते हैं:

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

तब हम अपने मॉडल को प्रशिक्षित कर सकते हैं:

In [19]:
trainer.train()

  0%|          | 31/7002 [04:16<14:27:52,  7.47s/it]

एक बार प्रशिक्षण पूरा हो जाने पर, हम अपने मॉडल का मूल्यांकन कर सकते हैं और सत्यापन सेट पर इसकी उलझन इस प्रकार जान सकते हैं:

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

## मास्क लैंग्वेज मॉडलिंग (एमएलएम)

मास्क्ड लैंग्वेज मॉडल (एमएलएम) हम एक अतिरिक्त चरण के साथ पहले की तरह ही डेटासेट प्रीप्रोसेसिंग का उपयोग करेंगे:

हम कुछ वर्णों को बेतरतीब ढंग से "मास्क" करेंगे (उन्हें "[MASK]" से बदल देंगे) और लेबल को इस प्रकार समायोजित करेंगे कि लेबल केवल "[MASK]" स्थिति में हों (क्योंकि हमें उन वर्णों की भविष्यवाणी करने की आवश्यकता नहीं है जो "मास्क" नहीं हैं ).

इस उदाहरण में, हम [`distilroberta-base`](https://huggingface.co/distilroberta-base) मॉडल का उपयोग करेंगे। आप [यहां](https://huggingface.co) सूचीबद्ध किसी भी चेकपॉइंट को चुन सकते हैं/मॉडल?फ़िल्टर=कारण-एलएम):

In [ ]:
model_checkpoint = "distilroberta-base"

हम पहले की तरह ही टोकननाइज़र फ़ंक्शन लागू कर सकते हैं, हमें बस अपने द्वारा चुने गए चेकपॉइंट का उपयोग करने के लिए अपने टोकननाइज़र को अपडेट करना होगा:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

पहले की तरह, हम पाठों को एक साथ समूहित करते हैं और उन्हें लंबाई `ब्लॉक_साइज़` के नमूनों में विभाजित करते हैं, यदि आपके डेटासेट में अलग-अलग वाक्य हैं, तो आप इस चरण को छोड़ सकते हैं।

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

बाकी सब कुछ वैसा ही है जैसा हमने पहले किया था, दो अपवादों को छोड़कर, पहले हम नकाबपोश भाषा मॉडलिंग के लिए उपयुक्त मॉडल का उपयोग करते हैं:

In [ ]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

दूसरे, हम एक विशेष डेटा_कोलेटर का उपयोग करते हैं। डेटा_कोलेटर एक फ़ंक्शन है जो नमूने लेने और उन्हें टेंसर में बैच करने के लिए जिम्मेदार है।

पिछले उदाहरण में, हमारे पास करने के लिए कुछ खास नहीं था, इसलिए हमने इस पैरामीटर के लिए केवल डिफ़ॉल्ट मान का उपयोग किया था। यहां हम यादृच्छिक "MASK" कर रहे हैं।

हम इसे प्रीप्रोसेसिंग चरण (`टोकनाइज़र`) के रूप में कर सकते हैं, लेकिन प्रत्येक चरण में वर्णों को हमेशा उसी तरह से मास्क किया जाता है, डेटा_कोलेटर में इस चरण को करके, हम यह सुनिश्चित कर सकते हैं कि यादृच्छिक मास्किंग होहर बार जब हम डेटा की जांच करते हैं तो यह एक नए तरीके से किया जाता है।

मास्किंग को लागू करने के लिए, `ट्रांसफॉर्मर्स` मास्क किए गए भाषा मॉडल के लिए `डेटाकोलेटरफॉरलैंग्वेजमॉडलिंग` प्रदान करता है। हम मास्किंग की संभावना को समायोजित कर सकते हैं:

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

फिर हमें सब कुछ प्रशिक्षक को सौंपना होगा और प्रशिक्षण शुरू करना होगा:

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

In [ ]:
trainer.train()

पहले की तरह, हम सत्यापन सेट पर अपने मॉडल का मूल्यांकन कर सकते हैं।

सीएलएम उद्देश्य की तुलना में उलझन बहुत कम है क्योंकि एमएलएम उद्देश्य के लिए हमें शेष टोकन तक पहुंच रखते हुए केवल छिपे हुए टोकन (यहां कुल का 15%) पर भविष्यवाणियां करने की आवश्यकता है।

इसलिए, मॉडल के लिए यह एक आसान काम है।

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
不要忘记将你的模型[上传](https://huggingface.co/transformers/model_sharing.html)到[🤗 模型中心](https://huggingface.co/models)。